## Data Extraction Using Mixtral:7x8b

**This is a data extraction project in which we extract product attributes of individual product categories one by one.**
**Problem**: 
- There is a large set of HTML text in an excel sheet that has all the product attributes of our products. <br>
- The product attributes are not individually listed anywhere. <br>
- We need a dataset of product attributes in which various properties of each product are listed separately, in order to build a PIM system, and to use specific attributes to SEO-optimize our webshop texts. <br>
- The Challenge is that the text is not regular, different attributes come in a variety of formats from one product to the next. <br>
- Another challenge is that we do not have a comprehensive list of product properties and that also needs to be created along the way. <br>

**Solution**: 
1. The first solution was Text mining using Regular expressions. It was implemented on one product group by reading and analyzing product descriptions of many products and finding attributes and then generating the regex patterns to extract them. <br>
Successful, but took a lot of time and energy. 
2. The second solution was to use SpaCy and NLP methods to extract adjective and prepositional groups such as "mit Griff" or "mit Deckel" to then use them for attribute generation. <br>
This was a faster method than raw text mining, but the problem was a large number of false positives (adjectives that were not product attributes) and false negatives <br>
(items that were not adjectives or prepositional groups but were attributes of the product nevertheless). 
3. This lead to trying out LLMs. The first attempt was with the transformers library. However, running into Langchain and Ollama, I found them to be faster solutions. <br>
I used Ollama because it supported the Mixtral:7x8b model which is both compatible with structured outputs and also supports German language. <br>
The result is the following code that functions much better in extracting all the relevant data required for our products.

- Note: The prompt was an important part of the modeling which resulted in correct and coherent results that could be further processed.

In [1]:
## Importing libraries
import pandas as pd
import re
import json
from values import *
import ollama
import warnings

warnings.filterwarnings('ignore')

In [2]:
## Loading Productgroup Number and its name
val = Values()
wg_list = val.wr_gr
wg_name = val.wr_name

In [3]:
## loading the csv file that has the product information for each warengruppe (previously generated using wr_folder_building.ipynb)
file = pd.read_csv(f'{val.parent_dir}{wg_list}/{wg_list}.csv',delimiter=';',encoding='utf-8')
mined_text = file.copy()

In [4]:
## Extracting the sets from the items to process them later
mined_text['TEILIG'] = mined_text['BESCHREIBUNG'].str.extract(r'( *\d+ *-*tlg.|\d+-teilig|\d+tlg.*|\d+-*er *-*set|\d+ set|\d+ ?ply)',flags=re.IGNORECASE)
mined_text = mined_text[mined_text['TEILIG'].isna() == True]
mined_text = mined_text[['NUMMER','NAME','BESCHREIBUNG']]
# mined_text = mined_text[:10]

In [10]:
## Defining the prompt variables

allowed_keys = [   "NUMMER", # Produkt ID
    "NAME", # Produkt NAME
    "MATERIAL", # Kunststoff
    "ART", # Weinglas
    "STUECK", # 2 Stück
    "DESIGN", # Stillvolles Design
    "BREITE", # 12 cm
    "TIEFE", # 10 cm
    "HOEHE", # 7 cm
    "ANDERE-ABMESSUNGEN", # 8 cm
    "GEWICHT", # 50 kg
    "VOLT", # 230V
    "WATT", # 200W
    "KABEL", # 1 m
    "BRAND", # Hagen Grote
    "QUALITAET", # hochwertiger Edelstahl,
    "ANWENDUNG", # Eisbereiter
    "ANDERE-EIGENSCHAFTEN" 

]
example = { "NUMMER" : "111HP06",
    "NAME": "Eisbereiter wandelt Brotbackautomaten in Eis- und Sorbetmaschine", # Produkt NAME
    "MATERIAL": "Kunststoff", #  Kunststoff
    "ART": "Weinglas", # Weinglas
    "STUECK": "2", # 2 Stück
    "DESIGN":"Stillvolles Design", # Stillvolles Design
    "BREITE":"12 cm", # 12 cm
    "TIEFE":"10 cm", # 10 cm
    "HOEHE":"7 cm", # 7 cm
    "ANDERE-ABMESSUNGEN": "40 X 60 CM Tabletten", # 8 cm
    "GEWICHT": "50 kg", # 50 kg
    "VOLT":"230V", # 230V
    "WATT":"200W", # 200W
    "KABEL":"1 m", # 1 m
    "BRAND": "Hagen Grote",
    "QUALITAET": "hochwertiger Edelstahl", # hochwertiger Edelstahl
    "ANWENDUNG": "Eisbereiter", # Eisbereiter
    "ANDERE-EIGENSCHAFTEN": []
    }

In [11]:
## Check if all the keys in the list match those in the example (inconsistencies lead to poor results)
for item in allowed_keys:
    if item in example.keys():
        continue
    else:
        print(f"{item} not in example.")

In [12]:
## function to clean responses from the LLM
def clean_response(response_text):
    response_text = response_text.replace(r"\\u00fc","ü")
    try:
        response_dict = json.loads(response_text)
        cleaned_dict = {key: value.strip() if isinstance(value, str) else value for key, value in response_dict.items()}
        return cleaned_dict

    except json.JSONDecodeError :

        print(f"Error: The response is not a valid JSON object.")
        print(response_text)

        return None

In [13]:
### Loading the prompt and generating responses based on previous variables as well as extact instrctions
uncleaned_data = []

for id,name, beschreibung in zip(mined_text['NUMMER'],mined_text['NAME'],mined_text['BESCHREIBUNG']):
    print(id,name)

    prompt = f"""[INST]1. Bitte extrahieren Sie die Produktattribute aus dem folgenden Beschreibung und geben Sie diese als ein gültiges JSON-Objekt in deutscher Sprache aus.
                    2. Verwenden Sie NUR die {allowed_keys} Schlüssel und ändern Sie diese nicht.
                    3. Verwenden Sie ALLE erlaubten Schlüssel, um das JSON-Objekt zu formatieren, und wenn es keine Werte für die spezifischen Schlüssel im JSON-Objekt gibt, lassen Sie deren Werte leer.
                    4. Beschränken Sie die Werte auf maximal 50 Zeichen. 
                    5. Füllen Sie nur die Informationen aus, die ausdrücklich im Beschreibung erwähnt werden.
                    7. die json-Datei sollte keine ascii-Kodierung haben. Die Sprache ist deutsch und die entsprechende Kodierung sollte verwendet werden.
                    8. Überprüfen Sie, ob die Antwort ein JSON-Objekt ist, und wenn nicht, ändern Sie sie so, dass sie ein JSON-Objekt wird.
                    9. Überprüfen Sie doppelt, ob die Schlüssel alle genau gleich sind. 
                    10. nur und ausschließlich das JSON-Objekt ausgeben und sonst nichts. kein Text und keine Erklärung, nur das JSON-Objekt.
                    11. deutsche Zeichen richtig kodieren und dekodieren [äëöüß] sowohl in Groß- als auch in Kleinschreibung . Verwenden Sie stattdessen KEINE Ascii-Schlüssel.


                    Beispiel-RESPONSE: {json.dumps(example,ensure_ascii=False)}
                    [/INST]
                    
                    NUMMER: {id}
                    NAME: {name}
                    Beschreibung: {beschreibung}
        """

# Generate response using Mixtral 7x8b
    response = ollama.generate(model='mixtral:latest', prompt=prompt)

# Clean up the response
    uncleaned_data.append(response['response'])



021G10 Batterien, Typ AA, 1,5 V, 4 Stück
845C12 Batterie Knopfzelle CR2032, 2 Stück
760L03 Ersatzbürste für Mehrflächenreiniger 760L01
760L05 Ersatzbezüge zu 2-in-1 Saug- und Dampfreiniger, für Saugen und Wischen in einem Arbeitsgang
391R01 Elektro-Power-Grillrostreiniger: Rotierende Bürsten säubern eingebrannten Schmutz
782X03 Spin Mop: 100 Rotationen pro Minute reinigen, schrubben, polieren
862Z01 Saug- und Wischroboter: Saubere Wohnung auf Knopfdruck
823B02 3 Ersatzfilter für Artikel 823b01
883B01 Vibrations-Schrubber: Putzt, reinigt, poliert kraftvoll und gründlich
883B02 Ersatz-Bürsten zu Vibrations-Schrubber
883B03 Ersatz-Pads zu Vibrations-Schrubber
843B01 Mobile Akku-Lampe: Licht, wo immer Sie es brauchen, Küchenlampe, Küchenlampe LED, Küchenlampe Decke
863F02 Mobile Mini-LED-Leuchten mit Bewegungssensor, Schrankbeleuchtung, batteriebetrieben, LED-Licht
863F03 Mobile Mini-LED-Leuchten mit Fernbedienung, Schrankbeleuchtung, batteriebetrieben, LED-Licht
744G01 Dampfreiniger: Mit 

In [14]:
## cleaning one step further and appending responses to a new list
new_data = []
for id,item in enumerate(uncleaned_data):

    new_data.append(clean_response(item))
    

Error: The response is not a valid JSON object.
 {
"NUMMER": "391R01",
"NAME": "Elektro-Power-Grillrostreiniger: Rotierende Bürsten säubern eingebrannten Schmutz",
"MATERIAL": "",
"ART": "Grillreiniger",
"STUECK": "",
"DESIGN": "",
"BREITE": "",
"TIEFE": "",
"HOEHE": "28 x 8 x 10 cm (B/T/H)",
"ANDERE-ABMESSUNGEN": "",
"GEWICHT": "350 g",
"VOLT": "",
"WATT": "",
"KABEL": "",
"BRAND": "",
"QUALITAET": "schlagfester Hartkunststoff-Korpus",
"ANWENDUNG": "Grillreiniger",
"ANDERE-EIGENSCHAFTEN": [""],
}


In [15]:
## saving clean json-objects to a json file
with open(f'{val.parent_dir}{val.wr_gr}/{val.wr_name}_{val.wr_gr}.json', 'w', encoding="utf-8") as f:
    json.dump(new_data, f, indent=4, ensure_ascii=False)    